In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.autograd import Variable
from sklearn.metrics import r2_score
from sklearn import preprocessing
#import scipy.io as sio

from models import AE
from models import DNN
import numpy as np
import pandas as pd

In [2]:
from scipy.stats import pearsonr

# Parameters

In [3]:
# Define parameters
epochs = 500 #200,500,1000
dim_au_in = 20049
dim_au_out = 512 #8, 16, 32, 64, 128, 256,512
dim_dnn_in = dim_au_out
dim_dnn_out=1

# Import data

In [4]:
data_r=pd.read_csv('data/GDSCexpression.csv',index_col=0)
label_r=pd.read_csv('data/GDSClabel.csv',index_col=0)

In [5]:
label_r=label_r.fillna(1)

# Your is gene-cell, mine is cell-gene

In [6]:
len(set(data_r.index))

727

In [7]:
data = data_r
label = label_r.iloc[:,10]
scaler = preprocessing.StandardScaler(with_mean=True, with_std=True)
data = scaler.fit_transform(data)

In [8]:
print(np.std(data))
print(np.mean(data))

0.9999999999999997
6.316789844568889e-19


In [9]:
data.shape

(789, 11833)

In [10]:
label_r.shape

(789, 139)

# Split test train

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(data, label, test_size=0.2, random_state=42)

In [12]:
print(data.shape)
print(label.shape)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(789, 11833)
(789,)
(631, 11833) (631,)
(158, 11833) (158,)


In [13]:
print(X_train.max())
print(X_train.min())

23.78951596267762
-17.277718272630676


In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)
torch.cuda.set_device(device)

cuda:0


# Add all data to AE

In [15]:
trainData = torch.FloatTensor(X_train).to(device)
testData = torch.FloatTensor(X_test).to(device)
y = torch.FloatTensor(Y_train.values).to(device)
allData = torch.FloatTensor(data).to(device)

# construct TensorDataset
train_dataset = TensorDataset(trainData, trainData)
test_dataset = TensorDataset(testData, testData)
all_dataset = TensorDataset(allData, allData)

trainDataLoader1 = DataLoader(dataset=train_dataset, batch_size=200, shuffle=False)
trainDataLoaderall = DataLoader(dataset=all_dataset, batch_size=200, shuffle=True)

# The model

In [16]:
# Initialization

#autoencoder = AE(dim=train_dataset.shape[1]).to(device)
autoencoder = AE(dim_au_in = X_train.shape[1],dim_au_out=dim_au_out).to(device)
optimizer = optim.Adam(autoencoder.parameters(), lr=1e-3)
#loss1
loss_func = nn.SmoothL1Loss().to(device)
#loss2
#loss_func = nn.BCELoss()
loss_train = np.zeros((epochs, 1))

In [ ]:
# Train autoencoder

for epoch in range(epochs):
    # 不需要label，所以用一个占位符"_"代替
    for batchidx, (x, _) in enumerate(trainDataLoaderall):
        x.requires_grad_(True)
        # encode and decode 
        decoded, encoded = autoencoder(x)
        # compute loss
        print(encoded.shape, decoded.shape)
        loss = loss_func(decoded, x)      
        # update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
           
    loss_train[epoch,0] = loss.item()  
    print('Epoch: %04d, Training loss=%.8f' %
          (epoch+1, loss.item()))

torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0001, Training loss=0.37606168
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0002, Training loss=0.37618503
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0003, Training loss=0.38026953
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0004, Training loss=0.37628207
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Siz

torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0037, Training loss=0.34800720
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0038, Training loss=0.35093251
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0039, Training loss=0.34695533
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0040, Training loss=0.34799850
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Siz

torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0073, Training loss=0.28339151
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0074, Training loss=0.28571820
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0075, Training loss=0.28651392
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0076, Training loss=0.28337553
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Siz

torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0109, Training loss=0.27402315
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0110, Training loss=0.28037822
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0111, Training loss=0.27386183
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0112, Training loss=0.27344003
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Siz

Epoch: 0144, Training loss=0.27479759
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0145, Training loss=0.26555079
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0146, Training loss=0.26768562
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0147, Training loss=0.27058223
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0148, Training loss=0.26932007
torch.Size([200, 512]) torch.Size([200, 11

...

Epoch: 0288, Training loss=0.26124454
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0289, Training loss=0.26531699
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0290, Training loss=0.25956339
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0291, Training loss=0.26183161
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0292, Training loss=0.25814167
torch.Size([200, 512]) torch.Size([200, 11

Epoch: 0324, Training loss=0.25609401
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0325, Training loss=0.25690749
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0326, Training loss=0.25436890
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0327, Training loss=0.25794801
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0328, Training loss=0.25780854
torch.Size([200, 512]) torch.Size([200, 11

torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0361, Training loss=0.25812876
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0362, Training loss=0.25625533
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0363, Training loss=0.25741038
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0364, Training loss=0.25149128
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Siz

torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0397, Training loss=0.25833881
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0398, Training loss=0.25483537
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0399, Training loss=0.25413957
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0400, Training loss=0.25788233
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Siz

torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0433, Training loss=0.24858469
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0434, Training loss=0.25645795


In [ ]:
torch.save(autoencoder.state_dict(), 'saved/models/AE_GDSC.pkl')

In [ ]:
# extract features
_, encodedTrainData = autoencoder(trainData)
featureTensor = encodedTrainData.double()
feature = featureTensor.detach().cpu().numpy()

In [ ]:
print(feature.shape)

In [ ]:
feature

In [ ]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1)
clf.fit(feature, Y_train.values)

In [ ]:
_,testFeature = autoencoder(testData)
lasso = clf.predict(testFeature.detach().cpu().numpy())

In [ ]:
r2_score(lasso,Y_test)

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(feature)

In [ ]:
tsne_results

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.scatter(tsne_results[:, 0], tsne_results[:, 1], label="label")
plt.legend()
plt.savefig("saved/figures/tsne_ae_gdsc.png")

In [ ]:
EPOCH = 500

In [ ]:
# Load data
# data type conversion
B_feature = torch.FloatTensor(feature).to(device)
y = torch.FloatTensor(Y_train.values).to(device)
# construct TensorDataset
b_data = TensorDataset(B_feature, y)
trainDataLoader2 = DataLoader(dataset=b_data, batch_size=200, shuffle=False)

In [ ]:
b_data.tensors

In [ ]:
b_data

In [ ]:
dim_dnn_in

In [ ]:
# Initialization DNN model

predictor = DNN(dim_dnn_in, dim_dnn_out).to(device)
optimizer = optim.Adam(predictor.parameters(), lr=1e-3,betas=(0.9,0.99))
#loss1-softmax
loss_func = nn.MSELoss().to(device)
#loss2-sigmoid
#loss_func = nn.BCELoss().to(device)
#loss3-sigmoid
#loss_func = nn.CrossEntropyLoss()

#criterion = torch.nn.MSELoss(size_average=True)
#criterion = torch.nn.BCELoss(size_average=True) # Defined loss function
#optimizer = optim.Adm(model.parameters(), lr=0.01) # Defined optimizer
loss_train = np.zeros((epochs, 1))

In [ ]:
# train model
for epoch in range(EPOCH):
    print('Epoch: ',epoch)
    for step,(batch_x,batch_y) in enumerate(trainDataLoader2):
        b_x = Variable(batch_x)
        b_y = Variable(batch_y)
        # predict label
        output = predictor(b_x)
        # b_y=F.sigmoid(b_y) 
        
        #print
        #print(output)
        #print(b_y)
        # compute loss
        loss = loss_func(output,b_y)
        #loss = criterion(output, b_y)
        
        # update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_train[epoch,0] = loss.item()  
    print('Epoch: %04d, Training loss=%.8f' %
          (epoch+1, loss.item())) 

# Save model
torch.save(predictor.state_dict(), 'saved/models/DNN_GDSC.pkl')

In [ ]:
# Get tesing feature
_,testFeature = autoencoder(testData)

In [ ]:
testpredict = predictor(testFeature)

In [ ]:
testpredict

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
r2_score(testpredict.detach().cpu().numpy(),Y_test)

In [ ]:
mean_squared_error(testpredict.detach().cpu().numpy(),Y_test)